Veres Melitta-Kinga, azonosító: 4447

In [1]:
/*1.SELECT utasítás segítségével adjuk meg minden kocsma esetén, hogy mennyire “népszerű” (KocsmaNev, Népszerűség) formában, ahol:
ha több, mint 4 barát kedveli, akkor: Népszerűség értéke: ‘Magas’
ha 2-3 barát kedveli, akkor: Népszerűség értéke: ‘Elfogadhato’
ha kevesebb, mint 2 barát kedveli, akkor: Népszerűség értéke: ‘Alacsony’*/

SELECT k.Nev AS KocsmaNev,
       CASE
           WHEN COUNT(ke.BaratID) >= 4 THEN 'Magas'
           WHEN COUNT(ke.BaratID) BETWEEN 2 AND 3 THEN 'Elfogadhato'
           ELSE 'Alacsony'
       END AS Nepszeruseg
FROM Kocsmak k
LEFT JOIN Kedvencek ke ON k.KocsmaID = ke.KocsmaID
GROUP BY k.Nev
        

Warning: Null value is eliminated by an aggregate or other SET operation.

(17 rows affected)

Batch execution time: 00:00:00.049

Total execution time: 00:00:00.049

KocsmaNev,Nepszeruseg
Bleriot,Magas
Bulgakov,Elfogadhato
E,Alacsony
El,Alacsony
Elb,Alacsony
ElbE,Alacsony
Hibrid,Alacsony
Insomnia,Alacsony
kkk,Alacsony
Klausen,Magas


In [4]:
/*2.Írjunk tárolt eljárást, melynek bemenő paramétere egy természetes szám (@pSzam), kimeneti paramétere: @pOut-int típusú! 
Ha @pSzam<0, írjunk ki megfelelő hibaüzenetet, valamint @pOut = -1! 
Ellenkező esetben, írassuk ki azon kocsmá(ka)t (KocsmaNev), mely(ek)ben kevesebb italtípusból válogathatunk, 
mint a paraméterként megadott szám (0 is lehet a szám)! Ekkor @pOut=a feltételnek eleget tevő kocsmák száma! 
Ha nincs egyetlen ilyen kocsma sem, írassunk ki megfelelő figyelmeztető üzenetet is!*/

CREATE OR ALTER PROCEDURE KocsmaItalok
    @pSzam INT,
    @pOut INT OUTPUT
AS
BEGIN
    IF @pSzam < 0
    BEGIN
        PRINT 'Hiba: pSzam negativ!'
        SET @pOut = -1
    END
    ELSE
    BEGIN
        SET @pOut = (SELECT COUNT(*) FROM (
            SELECT k.Nev
            FROM Kocsmak k
            JOIN Arak a ON k.KocsmaID = a.KocsmaID
            JOIN Italok i ON a.ItalID = i.ItalID
            JOIN ItalTipusok it ON i.TipusID = it.TipusID
            GROUP BY k.Nev
            HAVING COUNT(DISTINCT it.TipusID) < @pSzam
            )AS eredmeny)
        
        IF @pOut = 0
    BEGIN
            PRINT'Nincs olyan kocsma, ahol kevesebb italtipusbol valogathatunk, mint pSzam!';
    END
    ELSE
    BEGIN
        SELECT k.Nev AS KocsmaNev
        FROM Kocsmak k
        JOIN Arak a ON k.KocsmaID = a.KocsmaID
        JOIN Italok i ON a.ItalID = i.ItalID
        JOIN ItalTipusok it ON i.TipusID = it.TipusID
        GROUP BY k.Nev
        HAVING COUNT(DISTINCT it.TipusID) < @pSzam

        SET @pOut=@@ROWCOUNT;
        PRINT CAST(@pOut AS NVARCHAR(10)) + ' kocsma felelt meg az elvasaroknak'
    END
    END
END   

GO
    
DECLARE @result INT;

EXEC KocsmaItalok @pSzam = 2, @pOut = @result;






Commands completed successfully.

Batch execution time: 00:00:00.004

(7 rows affected)

7 kocsma felelt meg az elvasaroknak

Batch execution time: 00:00:00.035

Total execution time: 00:00:00.039

KocsmaNev
Bleriot
E
El
Elb
ElbE
kocsma
lll


In [60]:
/*3.Írjunk tárolt eljárást, mely a paraméterként megadott kocsma(Kocsma.Nev) 
címéből és italaibol eltűnteti a szóközöket és visszatériti az eltüntetett szóközök számát!*/

/*temporalis tablanal nem muszaj deklaralni csak legyen ott a neve elott a duplakereszt*/

CREATE  OR ALTER PROCEDURE Szokoz
    @KocsmaNev NVARCHAR(20)
AS
BEGIN
    DECLARE @szokozszam INT=0;


    UPDATE Kocsmak
    SET Cim = REPLACE(Cim, ' ', '')
    WHERE Nev = @KocsmaNev;

    SET @szokozszam = @szokozszam + LEN(Kocsmak.Cim) - LEN(REPLACE(Kocsmak.Cim, ' ', ''));


    UPDATE Italok
    SET ItalNev = REPLACE(ItalNev, ' ', '')
    FROM Italok i
    INNER JOIN Arak a ON i.ItalID=a.ItalID
    INNER JOIN Kocsmak k ON a.KocsmaID = k.KocsmaID
    WHERE k.Nev = @KocsmaNev
    

    SET @szokozszam = @szokozszam + LEN(Italok.ItalNev) - LEN(REPLACE(Italok.ItalNev, ' ', ''));


    SELECT @szokozszam AS Eltavolitott_szokozok_szama;
    
END

GO
EXEC Szokoz @KocsmaNev='vvv'


/*Ez egy helyes megoldas:
CREATE OR ALTER PROCEDURE SzokozEltuntetes
	@KocsmaNev NVARCHAR(20),
	@eltuntDb INT OUTPUT
AS
BEGIN
   
	DECLARE @eltuntDb1 INT;
	DECLARE @eltuntDb2 INT;
	DECLARE @cim nvarchar(50);

	SET @eltuntDb = 0;

	IF EXISTS (SELECT Cim FROM Kocsmak WHERE Nev = @KocsmaNev)
	BEGIN
		SET @cim = (SELECT Cim FROM Kocsmak WHERE Nev = @KocsmaNev);
		SELECT @eltuntDb1 = DATALENGTH(@cim)/2 - LEN(REPLACE(@cim, ' ', ''));
		UPDATE Kocsmak
		SET Cim = REPLACE(Cim, ' ', '')
		WHERE Nev = @KocsmaNev;
      
		SELECT @eltuntDb2 = LEN(I.ItalNev) - LEN(REPLACE(I.ItalNev, ' ', ''))
		FROM Italok I
		WHERE I.ItalID IN (SELECT A.ItalID FROM Arak A WHERE A.KocsmaID = (SELECT KocsmaID FROM Kocsmak WHERE Nev = @KocsmaNev));
		UPDATE Italok
		SET ItalNev = REPLACE(I.ItalNev, ' ', '')
		FROM Italok I
		WHERE I.ItalID IN (SELECT A.ItalID FROM Arak A WHERE A.KocsmaID = (SELECT KocsmaID FROM Kocsmak WHERE Nev = @KocsmaNev));
	
		
		--PRINT DATALENGTH(@cim)/2
		--PRINT LEN(REPLACE(@cim, ' ', ''))
		--PRINT @eltuntDb1
		--PRINT @eltuntDb2

		SET @eltuntDb = @eltuntDb1 + @eltuntDb2;
	END
	ELSE
	BEGIN
		--PRINT 'A kocsmanév nem létezik.';
		RAISERROR (N'A kocsmanév nem létezik.', 16, 1);
	END
END

DECLARE @eltuntDb INT;
EXEC SzokozEltuntetes @KocsmaNev = 'Bleriot', @eltuntDb = @eltuntDb OUTPUT;
PRINT 'Eltávolított szóközök száma: ' + CAST(@eltuntDb AS NVARCHAR(10));*/




Commands completed successfully.

Batch execution time: 00:00:00.025

(1 row affected)

UPDATE Italok executed

(1 row affected)

...

(1 row affected)

UPDATE Italok executed

Batch execution time: 00:00:00.034

Total execution time: 00:00:00.059

Eltavolitott_szokozok_szama
0


In [5]:
/*4.Írjunk tárolt eljárást, melynek bemenő paramétere @pAtlagAr - int típusú. 
A tárolt eljárás azon kocsmák bevételét (Kocsmak.Bevetel) csökkenti (UPDATE) 10%-kal, ahol az italok átlagára <=@pAtlagAr! Kimeneti paraméter: 
@pmodkt (int) - módosított kocsmák száma. 
Megj. Ha van olyan kocsma, melynek bevétele eleve 39999 EURO feletti, ezek esetén ne végezze el a módosítást!*/

CREATE OR ALTER PROCEDURE Tizszazalek
    @pAtlagAr INT,
    @pmodkt INT OUTPUT
AS
BEGIN
    CREATE TABLE #ModositottKocsmak (
        KocsmaID INT,
        Nev VARCHAR(20),
        Cim VARCHAR(30),
        Bevetel INT
    );

    INSERT INTO #ModositottKocsmak (KocsmaID, Nev, Cim, Bevetel)
    SELECT KocsmaID, Nev, Cim, Bevetel
    FROM Kocsmak
    WHERE Bevetel < 39999 AND Nev IN (
        SELECT k.Nev
        FROM Arak a
        JOIN Kocsmak k ON a.KocsmaID = k.KocsmaID
        GROUP BY k.Nev
        HAVING AVG(a.Ar) < 5
    );

    UPDATE Kocsmak
    SET Bevetel = Bevetel - Bevetel * 0.1
    WHERE Bevetel < 39999 AND Nev IN (
        SELECT k.Nev
        FROM Arak a
        JOIN Kocsmak k ON a.KocsmaID = k.KocsmaID
        GROUP BY k.Nev
        HAVING AVG(a.Ar) < 5
    );

    SET @pmodkt = @@ROWCOUNT;
	
    SELECT * FROM #ModositottKocsmak;

    DROP TABLE #ModositottKocsmak;
END

GO
DECLARE @pmodkt2 INT
EXEC Tizszazalek @pAtlagAr=5, @pmodkt=@pmodkt2 OUTPUT
PRINT CAST(@pmodkt2 AS NVARCHAR(10)) + '- modositott kocsmak szama'

Commands completed successfully.

Batch execution time: 00:00:00.006

(1 row affected)

(1 row affected)

(1 row affected)

1- modositott kocsmak szama

Batch execution time: 00:00:00.053

Total execution time: 00:00:00.059

KocsmaID,Nev,Cim,Bevetel
4,Soviet,ismeretlen,28066


In [6]:
/*5.Írj tárolt eljárást amely frissiti egy kocsma(Parameter: @pKocsmaNev) 
árait egy bizonyos ital tipusban(@pItalTipus), a piacon legalacsonyabb értékekkel 
vagyis minden ital ára a legkisebb legyen az illető kategóriából. */

CREATE OR ALTER PROCEDURE Arfrissites
@pKocsmaNev VARCHAR(20),
@pItalTipus VARCHAR(10)
AS
BEGIN
	UPDATE Arak
	SET Ar=(SELECT MIN(Ar) FROM Kocsmak k
	JOIN Arak a ON a.KocsmaID=k.KocsmaID
	JOIN Italok i ON a.ItalID=i.ItalID
	JOIN ItalTipusok it ON i.TipusID=it.TipusID
	WHERE k.Nev=@pKocsmaNev and it.TipusNev=@pItalTipus)
	WHERE KocsmaID = (SELECT KocsmaID FROM Kocsmak WHERE Nev = @pKocsmaNev)
	AND ItalID IN (SELECT ItalID FROM Italok WHERE TipusID = (SELECT TipusID FROM ItalTipusok WHERE TipusNev = @pItalTipus));

END

GO
EXEC Arfrissites @pKocsmaNev='Krajczar', @pItalTipus='sor'

Commands completed successfully.

Batch execution time: 00:00:00.003

(5 rows affected)

Batch execution time: 00:00:00.029

Total execution time: 00:00:00.032

In [7]:
/*6.Írjunk tárolt eljárást, melynek segítségével egy új kocsmát vezethetünk fel az adatbázisba! 
Bemenő paraméterek: @pKocsmaNev, @pCim, @pNyitasDatuma, @pBaratNev, @pBaratEmail, @pItalTipNev, @pItalNev, @pAr)! 
Ellenőrizzük, hogy a megadott kocsma létezik-e már az adatbázisban. 
Ha igen, írassunk ki hibaüzenetet és térítsünk vissza -1-t! 
Ellenkező esetben szúrjuk be a Kocsmák, Kedvencek és Arak táblákba a megfelelő sorokat! 
Fontos: Ha nem létezik az illető barát, ital, italtípus az adatbázisban, akkor először a 
Baratok, ItalTipusok, illetve Italok táblákba szúrjuk be a megfelelő adatokat!*/

CREATE OR ALTER PROCEDURE Kocsmabeszuras
@pKocsmaNev VARCHAR(20),
@pCim VARCHAR(30),
@pNyitasDatuma DATE,
@pBaratNev VARCHAR(30),
@pBaratEmail VARCHAR(20),
@pItalTipNev VARCHAR(10),
@pItalNev VARCHAR(20),
@pAr REAL

AS
BEGIN
    DECLARE @vUjKocsmaID INT;
    DECLARE @vUjItalID INT;
    DECLARE @vUjItalTipID INT

    DECLARE @Results TABLE (
        Nev VARCHAR(20),
        Cim VARCHAR(30),
        NyitasDatuma DATE,
        BaratNev VARCHAR(30),
        BaratEmail VARCHAR(20),
        ItalTipusNev VARCHAR(10),
        ItalNev VARCHAR(20),
        Ar REAL
    );

    IF EXISTS (SELECT Nev FROM Kocsmak 
    WHERE Nev=@pKocsmaNev)
    BEGIN
        PRINT 'A megadott kocsma mar letezik az adatbazisban.'
        RETURN -1;
    END
    ELSE 
    BEGIN
        IF NOT EXISTS (SELECT Nev FROM Baratok
            WHERE Nev=@pBaratNev)
        BEGIN
            INSERT INTO Baratok(Nev, Email) VALUES (@pBaratNev, @pBaratEmail)
        END

        IF NOT EXISTS (SELECT TipusNev FROM ItalTipusok
            WHERE TipusNev=@pItalTipNev)
        BEGIN
            INSERT INTO ItalTipusok(TipusNev) VALUES (@pItalTipNev)
        END
        IF NOT EXISTS (SELECT ItalNev FROM Italok
            WHERE ItalNev=@pItalNev)
        BEGIN
            SET @vUjItalTipID=(SELECT TipusID FROM ItalTipusok
                                WHERE TipusNev=@pItalTipNev)
            INSERT INTO Italok(ItalNev, TipusID) VALUES (@pItalNev, @vUjItalTipID)    
        END
        INSERT INTO Kocsmak(Nev, Cim, NyitasDatuma) VALUES (@pKocsmaNev, @pCim, @pNyitasDatuma)

        SET @vUjKocsmaID= (SELECT KocsmaID FROM Kocsmak
                            WHERE Nev=@pKocsmaNev)
        
        SET @vUjItalID = (SELECT ItalID FROM Italok 
                            WHERE ItalNev = @pItalNev);

        INSERT INTO Arak (Ar, KocsmaID, ItalID)
        VALUES (@pAr, @vUjKocsmaID, @vUjItalID);

    
        INSERT INTO @Results (Nev, Cim, NyitasDatuma, BaratNev, BaratEmail, ItalTipusNev, ItalNev, Ar)
        VALUES(@pKocsmaNev, @pCim, @pNyitasDatuma, @pBaratNev, @pBaratEmail, @pItalTipNev, @pItalNev, @pAr)

        SELECT * FROM @Results;

    END
END

GO
EXEC KocsmaBeszuras 
@pKocsmaNev='E',
@pCim='Cim Hibrid',
@pNyitasDatuma='2023-11-10',
@pBaratNev='Vajk',
@pBaratEmail='vajk@yahoo.com',
@pItalTipNev='kave',
@pItalNev='espresso',
@pAr=10




    

Commands completed successfully.

Batch execution time: 00:00:00.052

A megadott kocsma mar letezik az adatbazisban.

Batch execution time: 00:00:00.030

Total execution time: 00:00:00.082

In [8]:
/*7.Írjunk függvényt, mely visszatéríti, hogy a paraméterként megadott nevű barát (@pBaratNev,@pTel) hány koncsmát kedvel azok közül, 
amelyekben árulnak @pItalNev nevű italt (@pItalNev-bemeneti paraméter)! 
Megj. Ellenőrizzük le, hogy az adott barát, ital létezik-e az adatbázisban (ha nem, térítsünk vissza -3-t).*/

CREATE FUNCTION dbo.BaratKedvencKocsmakSzama
(
    @pBaratNev VARCHAR(30),
    @pTel VARCHAR(10),
    @pItalNev VARCHAR(20)
)
RETURNS INT
AS
BEGIN
    DECLARE @BaratID INT
    DECLARE @ItalID INT

    SELECT @BaratID = BaratID
    FROM Baratok
    WHERE Nev = @pBaratNev AND Tel = @pTel

    IF @BaratID IS NULL
    BEGIN
        RETURN -3
    END

    SELECT @ItalID = ItalID
    FROM Italok
    WHERE ItalNev = @pItalNev

    IF @ItalID IS NULL
    BEGIN
        RETURN -3 
    END

    RETURN (
        SELECT COUNT(*)
        FROM Kedvencek AS k
        WHERE k.BaratID = @BaratID
        AND EXISTS (
            SELECT 1
            FROM Arak AS a
            WHERE a.KocsmaID = k.KocsmaID
            AND a.ItalID = @ItalID
        )
    )
END

GO

DECLARE @BaratNev VARCHAR(30) = 'Csaba'
DECLARE @Tel VARCHAR(10) = '0743565678'
DECLARE @ItalNev VARCHAR(20) = 'Unicum'
DECLARE @KedvencKocsmakSzama INT

SET @KedvencKocsmakSzama = dbo.BaratKedvencKocsmakSzama(@BaratNev, @Tel, @ItalNev) /*SELECT is lehet SET helyett*/

PRINT @BaratNev + ' ' + CAST(@KedvencKocsmakSzama AS VARCHAR) + ' kocsmat kedvel ahol aruljak a kovetkezo italt: ' + @ItalNev



: Msg 2714, Level 16, State 3, Procedure BaratKedvencKocsmakSzama, Line 5
There is already an object named 'BaratKedvencKocsmakSzama' in the database.

Batch execution time: 00:00:00.002

Csaba -3 kocsmat kedvel ahol aruljak a kovetkezo italt: Unicum

Batch execution time: 00:00:00.009

Total execution time: 00:00:00.011